In [5]:
import os
os.environ['http_proxy'] = 'http://127.0.0.1:7890'
os.environ['https_proxy'] = 'http://127.0.0.1:7890'

In [2]:
import torch
import torch.nn.functional as F
from copy import deepcopy

$$
 \dfrac{x_1 \cdot x_2}{\max(\Vert x_1 \Vert _2 \cdot \Vert x_2 \Vert _2, \epsilon)}
$$

In [6]:
F.cosine_similarity??

In [3]:
from sentence_transformers import SentenceTransformer, LoggingHandler, losses, InputExample, models
from torch.utils.data import DataLoader

model = SentenceTransformer('all-MiniLM-L6-v2', device='cpu')

[2024-02-10 19:40:51,465] [INFO] [real_accelerator.py:161:get_accelerator] Setting ds_accelerator to cuda (auto detect)


/home/whaow/anaconda3/lib/python3.10/site-packages/torch/_utils.py:776: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


## model

In [9]:
model

SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False})
  (2): Normalize()
)

- `model[0]`: `token_embeddings` 
- `model[1]`: `pooling_mode_mean_tokens`
    - `sentence_embedding`: 

- 其他的 Pooling 方法

```
pooling_model = models.Pooling(word_embed_model.get_word_embedding_dimension(), 
                       pooling_mode='cls',
                       pooling_mode_cls_token=True, 
                       pooling_mode_mean_tokens = False)
```

## dataloader

In [8]:
train_examples = [
    InputExample(texts=['This is a positive pair', 
                        'Where the distance will be minimized'], 
                 label=1),
#     InputExample(texts=['This is a negative pair', 'Their distance will be increased'], label=0)
]

In [9]:
# sentences input, 
train_dataloader = DataLoader(train_examples, shuffle=True, batch_size=2)

In [10]:
train_dataloader.collate_fn

<function torch.utils.data._utils.collate.default_collate(batch)>

In [12]:
train_dataloader.collate_fn = model.smart_batching_collate
batch = next(iter(train_dataloader))
batch 

([{'input_ids': tensor([[ 101, 2023, 2003, 1037, 3893, 3940,  102]]),
   'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0]]),
   'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1]])},
  {'input_ids': tensor([[  101,  2073,  1996,  3292,  2097,  2022, 18478,  2094,   102]]),
   'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0]]),
   'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1]])}],
 tensor([1]))

## losses

- $(a,b)$: pair sentences embeddings

$$
\frac12|a,b|^2, \ell=1\\
\text{ReLU}^2(\epsilon-|a,b|), \ell=0
$$

In [13]:
losses.ContrastiveLoss??

In [14]:
train_loss = losses.ContrastiveLoss(model=model)

In [18]:
# list(train_loss.named_parameters())

In [15]:
list(train_loss.named_parameters())[0][1].shape

torch.Size([30522, 384])

In [21]:
# SiameseDistanceMetric.COSINE_DISTANCE
# lambda x, y: 1-F.cosine_similarity(x, y)
train_loss.distance_metric

<function sentence_transformers.losses.ContrastiveLoss.SiameseDistanceMetric.<lambda>(x, y)>

## model.forward

In [23]:
batch

([{'input_ids': tensor([[ 101, 2023, 2003, 1037, 3893, 3940,  102]]),
   'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0]]),
   'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1]])},
  {'input_ids': tensor([[  101,  2073,  1996,  3292,  2097,  2022, 18478,  2094,   102]]),
   'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0]]),
   'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1]])}],
 tensor([1]))

In [22]:
batch[1]

tensor([1])

In [18]:
batch[0][0]['input_ids'].shape

torch.Size([1, 7])

In [24]:
features, labels = batch
features

[{'input_ids': tensor([[ 101, 2023, 2003, 1037, 3893, 3940,  102]]),
  'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0]]),
  'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1]])},
 {'input_ids': tensor([[  101,  2073,  1996,  3292,  2097,  2022, 18478,  2094,   102]]),
  'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0]]),
  'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1]])}]

In [61]:
# model[2](model[1](model[0](features[0])))

In [25]:
model

SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False})
  (2): Normalize()
)

In [26]:
feature_cpy = deepcopy(features)

In [22]:
model[1](model[0](features[0]))['sentence_embedding'][0, :5]

tensor([-0.2930,  0.3243, -0.6169, -0.0097, -0.1806], grad_fn=<SliceBackward0>)

In [23]:
(torch.sum(model[0](feature_cpy[0])['token_embeddings'], dim=1) / 7)[0, :5]

tensor([-0.2930,  0.3243, -0.6169, -0.0097, -0.1806], grad_fn=<SliceBackward0>)

## forward loss

In [28]:
sent1_embed = model(features[0])['sentence_embedding']
sent2_embed = model(features[1])['sentence_embedding']

In [29]:
# 1 - F.cosine_similarity(sent1_embed, sent2_embed)
train_loss.distance_metric(sent1_embed, sent2_embed)

tensor([0.9867], grad_fn=<RsubBackward1>)

In [30]:
1 - F.cosine_similarity(sent1_embed, sent2_embed)

tensor([0.9867], grad_fn=<RsubBackward1>)

In [31]:
train_loss(features, labels)

tensor(0.4868, grad_fn=<MeanBackward0>)

In [27]:
1/2*(1-F.cosine_similarity(sent1_embed, sent2_embed))**2

tensor([0.4868], grad_fn=<MulBackward0>)

In [ ]:
model.fit([(train_dataloader, train_loss)], show_progress_bar=True)

## Pooling methods

### pooling_mode_mean_tokens

In [32]:
train_examples = [
    InputExample(texts=['This is a positive pair', 'Where the distance will be minimized'], label=1),
#     InputExample(texts=['This is a negative pair', 'Their distance will be increased'], label=0)
]
# sentences input, 
train_dataloader = DataLoader(train_examples, shuffle=True, batch_size=2)
train_dataloader.collate_fn = model.smart_batching_collate
batch = next(iter(train_dataloader))
# batch 
features, labels = batch
features

[{'input_ids': tensor([[ 101, 2023, 2003, 1037, 3893, 3940,  102]]),
  'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0]]),
  'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1]])},
 {'input_ids': tensor([[  101,  2073,  1996,  3292,  2097,  2022, 18478,  2094,   102]]),
  'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0]]),
  'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1]])}]

In [33]:
word_embed_model = models.Transformer('bert-base-uncased')
# a pool function over the token embeddings
pooling_model = models.Pooling(word_embed_model.get_word_embedding_dimension(), 
                               pooling_mode_cls_token=False, 
                               pooling_mode_mean_tokens=True, 
                               pooling_mode_max_tokens=False, 
                               pooling_mode_mean_sqrt_len_tokens=False)
model = SentenceTransformer(modules=[word_embed_model, pooling_model])

In [46]:
model(features[0])['sentence_embedding'][0][:5]

tensor([-0.0863, -0.2668,  0.5492, -0.4936, -0.1374], grad_fn=<SliceBackward0>)

### cls pooling method

In [47]:
train_examples = [
    InputExample(texts=['This is a positive pair', 'Where the distance will be minimized'], label=1),
#     InputExample(texts=['This is a negative pair', 'Their distance will be increased'], label=0)
]
# sentences input, 
train_dataloader = DataLoader(train_examples, shuffle=True, batch_size=2)
train_dataloader.collate_fn = model.smart_batching_collate
batch = next(iter(train_dataloader))
# batch 
features, labels = batch
features

[{'input_ids': tensor([[ 101, 2023, 2003, 1037, 3893, 3940,  102]]),
  'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0]]),
  'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1]])},
 {'input_ids': tensor([[  101,  2073,  1996,  3292,  2097,  2022, 18478,  2094,   102]]),
  'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0]]),
  'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1]])}]

In [48]:
word_embed_model = models.Transformer('bert-base-uncased')
# a pool function over the token embeddings
pooling_model = models.Pooling(word_embed_model.get_word_embedding_dimension(), 
                               pooling_mode = 'cls',
                               pooling_mode_cls_token=True, 
                               pooling_mode_mean_tokens = False)
model = SentenceTransformer(modules=[word_embed_model, pooling_model])

In [49]:
model(features[0])['sentence_embedding'][0][:5]

tensor([-0.1775, -0.0474,  0.1351, -0.3242, -0.5006], grad_fn=<SliceBackward0>)

### cls pooling from scartch

In [50]:
train_examples = [
    InputExample(texts=['This is a positive pair', 'Where the distance will be minimized'], label=1),
#     InputExample(texts=['This is a negative pair', 'Their distance will be increased'], label=0)
]
# sentences input, 
train_dataloader = DataLoader(train_examples, shuffle=True, batch_size=2)
train_dataloader.collate_fn = model.smart_batching_collate
batch = next(iter(train_dataloader))
# batch 
features, labels = batch
features

[{'input_ids': tensor([[ 101, 2023, 2003, 1037, 3893, 3940,  102]]),
  'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0]]),
  'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1]])},
 {'input_ids': tensor([[  101,  2073,  1996,  3292,  2097,  2022, 18478,  2094,   102]]),
  'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0]]),
  'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1]])}]

In [36]:
model[0](features[0])['token_embeddings'].shape

torch.Size([1, 7, 768])

In [56]:
model[0](features[0])['token_embeddings'][0, 0][:5]

tensor([-0.1775, -0.0474,  0.1351, -0.3242, -0.5006], grad_fn=<SliceBackward0>)